In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 0

from fastai.structured import *
from fastai.column_data import *
np.set_printoptions(threshold=50, edgeitems=20)

import torchtext
from torchtext import vocab, data
from torchtext.datasets import language_modeling
import torch.distributed as dist 
from torch.multiprocessing import Process

from fastai.rnn_reg import *
from fastai.rnn_train import *
from fastai.nlp import *
from fastai.lm_rnn import *

import dill as pickle
import collections

import gc
import multiprocessing as mp
mgr = mp.Manager()
ns = mgr.Namespace()

from datetime import datetime

In [18]:
class M:
    sample = 1.0
    cores = 3
    no_cuda = True
    #cores = mp.cpu_count()-1
    format = 'csv'
    index = 'id'
    sort_var = 'item_description'
    # first process item_description then name
    text_vars = ['item_description', 'name']
    cat_vars = ['item_condition_id', 'brand_name', 'shipping']
    combine = {'item_description': ['name', 'item_description'], 'name': ['name']}
    split_cat_for_text = True
    nbrands = 1000
    split_cat = 0
    start_time = datetime.now()
    
    #category_names = ['category_name_'+str(i) for i in range(split_cat)]
    
    category_names = ['category_name']
    
    cat_vars += category_names
    range_vars = ['min', 'max']
    contin_vars = [f'len_{tv}' for tv in text_vars]
    #contin_vars += range_vars
    target = 'price'
    dep = 'price'
    features = [index]+text_vars+cat_vars+contin_vars
    min_freq = {'item_description': int(75), 'name': int(45*sample+5)}

    size = 1
    nfcore = 5
    end_train = False

USE_GPU = False if M.no_cuda else True

M.enc_bptt, M.enc_max_seq=[50, 20], [100, 30]
M.text_emb_sz = [70, 30]  # size of each embedding vector
M.enc_n_hid = [100, 50]   # number of hidden activations per layer
M.enc_n_layers = [1, 1]   # number of layers
M.opt_fn = partial(optim.Adam, betas=(0.7, 0.99))

M.text_seq_last, M.ignore_pad = False, True
M.lm_dropouti, M.lm_dropoute, M.lm_dropouth = 0.05, 0.02, 0.05 
M.lm_dropout, M.lm_wdrop = 0.05, 0.1
M.lm_opt_fn = partial(optim.Adam, betas=(0.7, 0.99))
M.lm_reg_fn = partial(seq2seq_reg, alpha=2, beta=1)

M.dropouti, M.dropoute, M.dropouth=0.065, 0.01, 0.02
M.wdrop=0.03
M.cat_emb_drop=0.04
M.fc_layers=[512, 256, 64]
M.fc_drops =[0.001, 0.01, 0.1]

#M.lrs=np.array([1.0e-3, 1.0e-3,1.0e-3,1.0e-3])
M.lrs = 0.01
M.bs = 256

In [3]:
def override_defs(module, fns):
    module = sys.modules[module]
    for fn in fns:
        setattr(module, fn, fn)
        sys.modules[module] = module
        from module import fn
    return 

def show(s='.'):
    sys.stdout.write(s)
    sys.stdout.flush()

def save_df(df, file=None, fidx=None, st=None, index=False):
    # fidx can be 0 
    if fidx!=None: file = f'{SCRATCH}/{st}_{fidx}.{M.format}'
    if file==None: show('******ERROR**** in save_df ')
    df.to_feather(file) if (M.format=='feather') else df.to_csv(file, index=index)
    return file

def read_df(file, processed=False):
    df = pd.read_feather(file) if (M.format=='feather') else pd.read_csv(file)
    if processed: 
        for tv in M.text_vars: df[tv] = df[tv].apply(eval)
    return df

def run_distributed(fn, fn_args, cores=None):
    if cores==None or cores > mp.cpu_count()-1: cores = mp.cpu_count()-1
    pool = mp.Pool(processes = cores)
    res = pool.starmap(fn, fn_args)
    show(' done\n')
    pool.close()
    pool.join()
    return res

# DF Processing

In [4]:
from spacy.symbols import ORTH
from spacy.pipeline import SentenceSegmenter

smpl_tok = spacy.load('en')
smpl_tok.pipeline.append(SentenceSegmenter(smpl_tok.vocab))

re_rm = re.compile(r'\s*\[\s*rm\s*\]\s*')
re_sp = re.compile(r'\s+')
smpl_tok.tokenizer.add_special_case('[rm]', [{ORTH: '[rm]'}])

def simple_tok(x):
    x = str(x).lower()
    x = re_rm.sub(' [rm] ', x)
    x = ''.join(filter(lambda el: el in string.printable, x))
    x = re_sp.sub(r' ', x)
    #x = ' '.join(x.split())
    ret = [tok.text for tok in smpl_tok.tokenizer(x)]
    return ret

def fix_brand_col(df, brands):
    idxs = set(df.loc[df['brand_name']=='missing'].index)
    ol = len(idxs)
    for i,b in enumerate(brands):
        if len(b.split()) > 1: 
            b1 = b.lower()
            splt = df['name'].loc[idxs].apply(lambda x: len(str(x).lower().split(b1)))
        else:
            splt = df['name'].loc[idxs].str.split(b).apply(len)    
        fix_idx = splt[splt >1].index
        df.loc[fix_idx, 'brand_name'] = b
        idxs -= set(fix_idx)
    show(f'{round((ol-len(idxs))*100/ol)}% ')
    return
    
def preprocess_df(df, fidx, st):
    df.loc[df['item_description']=='No description yet', 'item_description'] = 'missing'
    for c in ['name', 'category_name', 'brand_name', 'item_description']:
        df.loc[:, c] = df[c].fillna('missing')
    fix_brand_col(df, M.brands)
    if st=='train':
        df.drop(df[df['price'] < 3.0].index, inplace=True)
    
    df.reset_index(drop=True, inplace=True)
    show('.')
    return save_df(df, fidx=fidx, st=st)

def numericalize(str_tok, data_fld):
    return [data_fld.vocab.stoi[x] for x in str_tok]

def get_cat_col(df, col):
    if M.split_cat_for_text: 
        ret = ['<category_name>'] + df['category_name'].str.split("/")
    else:
        ret = ['<category_name>'] + df['category_name'].apply(lambda x: [x])
    return ret

def tokenize(df, combine, data_fld=None):
    ret = get_cat_col(df, combine[-1])
    for col in combine: 
        ret += [f"<{col}>"] + df[col].apply(simple_tok)
    ret += ['<eos>']
    if data_fld:
        ret = ret.apply(partial(numericalize, data_fld=data_fld))
    show('.')
    return ret 

def numericalize_text(dfile, combine_text_vars, data_fld_files):
    df = read_df(dfile)
    # numericalize in order
    for text_var in M.text_vars:
        data_fld = pickle.load(open(data_fld_files[text_var],'rb'))
        combine = combine_text_vars[text_var]
        ret = get_cat_col(df, text_var)
        for col in combine:
            ret += [f"<{col}>"] + df[col].apply(simple_tok)
        ret += ['<eos>']
        ret = ret.apply(partial(numericalize, data_fld=data_fld))
        # convert to string as feather does not support list
        df.loc[:,text_var] = ret.apply(str)
        show('.')
    return save_df(df, dfile)

def get_examples(files, flds):
    examples = []
    #flds = [fld for fld in data_flds]
    for f in files:
        show(f'{f}        ')
        df = read_df(f, processed=True)
            #examples += list(df.apply(lambda x: data.Example.fromlist(list(x[cols]), \
            #                                                         fields), axis=1))
        for i in range(len(df)):
            ex = data.Example()
            for col in flds: setattr(ex, col, df[col].iloc[i])
            #print(col, type(getattr(examples[i], col)))
            examples.append(ex)
    return examples
    
def split_col(x, fillna=str(np.nan)):
    n = 5
    x = x.split("/")
    return x+[fillna]*(n-len(x)) if len(x)<n else x[:n]

def add_cols(df):
    cn = M.category_names
    if M.split_cat > 0:
        df[cn[0]], df[cn[1]], df[cn[2]], df[cn[3]], df[cn[4]] = zip(*df['category_name'].apply(split_col))
    for i,tv in enumerate(M.text_vars):
        df[f'len_{tv}'] = df[tv].apply(len)  

In [5]:
class ProcessDataFrame:
    '''Only Sequential processes go here. 
    Distributed processes need to be defined outside to keep it light.'''
    def __init__(self, path, st, ref=None, n=10):
        self.path, self.split = path, st
        self.preprocessed = []
        self.current_processed = []
        if ref==None:
            self.mapper, self.nas, self.df_sample, self.y_range, self.brands = None, None, None, None, None
            self.data_flds = {}
        else: 
            self.mapper, self.nas, self.df_sample, self.y_range, self.brands = \
                                    ref.mapper, ref.nas, ref.df_sample, ref.y_range, ref.brands
            self.data_flds = ref.data_flds
        self.n, self.iter = n, 1
        self.missing_idx, self.idx = None, None

    def preprocess(self):
        df = pd.read_csv(self.path, sep='\t')
        df = df.sample(frac=M.sample) if self.iter==1 else df.loc[self.missing_idx]     
        
        if self.split=='train' and self.iter==1:
            counter=collections.Counter(df['brand_name'].loc[df['brand_name'].isnull() == False])
            M.brands = [b for b,n in counter.most_common(M.nbrands)]
        df.rename(columns={f'{self.split}_id' : 'id'}, inplace=True)
        #cores = mp.cpu_count()-1
        self.n = min(self.n, int(1+len(df)/10000))
        show(f'Pre-processing {self.split} iter {self.iter} ({self.n})')
        if self.iter==1:
            self.missing_idx = set(df['id'].copy())
            self.idx = set(df['id'].copy())
        dfa = np.array_split(df,self.n)
        file_ids = [f'{self.iter}{i}' for i in range(self.n)]
        self.current_processed = run_distributed(partial(preprocess_df, st=self.split), \
                                            list(zip(dfa, file_ids)), M.cores)
        self.preprocessed += self.current_processed
        self.preprocessed = list(set(self.preprocessed))
        show(' done\n')
        gc.collect()
        return 
    
    def build_vocab(self, text_var, combine, f=0.2, eos=[]):
        show(f' building vocab for {text_var}... ')
        f1 = np.sqrt(f)
        # f1 sample files and f1 samples from each so f1*f1
        df = pd.concat([read_df(file) for file in random.sample(self.preprocessed, int(round(f1*self.n)))])
        df = df.sample(frac=f1)
        text = pd.Series()
        dfa = np.array_split(df, self.n)
        for res in run_distributed(partial(tokenize, combine=combine), \
                            list(zip(dfa)), M.cores):
            text=text.append(res)
        data_fld = data.Field(sequential=True, use_vocab=False)
        data_fld.build_vocab(text, min_freq=M.min_freq[text_var])
        # pad-token is numericalized, used for dataloader iterator 
        data_fld.pad_token = data_fld.vocab.stoi[data_fld.pad_token]  
        show(f'{text_var} tokens {len(data_fld.vocab.itos[:])}\n')
        #outf = f'{SCRATCH}/{text_var}_field.pkl'
        #pickle.dump(data_fld, open(outf,'wb'))
        self.data_flds[text_var] = data_fld
        gc.collect()
        return
        
    def numericalize_text(self):
        show('Numericalizing ...')
        # star map does not handle passing of datafields 
        dff = {}
        for text_var in M.text_vars:
            outf = f'{SCRATCH}/{text_var}_field.pkl'
            pickle.dump(self.data_flds[text_var], open(outf,'wb'))
            dff[text_var] = outf
        run_distributed(partial(numericalize_text, combine_text_vars=M.combine, data_fld_files=dff), \
                                       list(zip(self.current_processed)), M.cores)
        gc.collect()
        return

    def postprocess_df(self):        
        show('Post processing ')
        ## takes 20sec. not dist required. 
        df = pd.concat([read_df(f, processed=True) for f in self.current_processed])
        if len(df) == 0 : return 
        
        add_cols(df)
        for v in M.cat_vars: df[v] = df[v].astype('category').cat.as_ordered()
        # keep a small copy to apply cat to test later
        if self.split == 'train' and self.iter==1: 
            self.df_sample = df.sample(2).copy()
        
        # make float continuous variables if any
        if self.split == 'test': df[M.dep] = 0
        for v in M.contin_vars+[M.dep]: df[v] = df[v].astype('float32')
        skip_flds = [c for c in df.columns if c not in M.cat_vars+M.contin_vars+[M.dep]]
        if self.split == 'train' and self.iter==1:
            self.cat_sz = [(c, len(df[c].cat.categories)+1) for c in M.cat_vars]
            self.cat_emb_szs = [(c, min(50, (c+1)//2)) for _,c in self.cat_sz]
            
            if len(M.contin_vars)==0:
                processed_df, y, nas = proc_df(df, M.dep, skip_flds=skip_flds, do_scale=False)
            else:
                processed_df, y, self.nas, self.mapper = proc_df(df, M.dep, skip_flds=skip_flds, do_scale=True)
                
            #convert to log scale
            processed_df[M.dep] = np.log(y+1)
            y_group = processed_df.groupby(M.cat_vars)[M.dep]
            y_range = pd.DataFrame({'min': y_group.agg('min'), 'max': y_group.agg('max')})
            self.y_range = save_df(y_range, f'{SCRATCH}/y_range.csv', index=True)
        else: 
            apply_cats(df, self.df_sample)
            
            # PROCESSED DF from train
            if len(M.contin_vars)==0:
                processed_df, _, nas = proc_df(df, M.dep, skip_flds=skip_flds, do_scale=False)
            else:
                processed_df, _, nas, mapper = proc_df(df, M.dep, do_scale=True, skip_flds=skip_flds,
                                  mapper=self.mapper, na_dict=self.nas)
            processed_df[M.dep] = df[M.dep]  
        
        y_range_df = read_df(self.y_range)
        processed_df[M.index] = df[M.index]
        processed_df[M.text_vars] = df[M.text_vars]
        processed_df = pd.merge(processed_df, y_range_df, how='left', on=M.cat_vars)
        processed_df['min'].fillna(min(y_range_df['min']), inplace=True)
        processed_df['max'].fillna(min(y_range_df['max']), inplace=True)
        
        df = processed_df
        self.missing_idx -= set(df[M.index])
        gc.collect()

        # shuffle and save
        dfa = np.array_split(df, self.n)
        run_distributed(save_df, list(zip(dfa, self.current_processed)), M.cores)
        gc.collect()
        return 
    
    def get_examples(self):
        examples = []
        # we'll need to keep ids as torch-text shuffles the data based on text length
        self.data_flds[M.index] = data.Field(sequential=False, tensor_type=torch.LongTensor, use_vocab=False)
        # text var already built
        #for tv in M.text_vars:
        #    data_fields[tv] = pickle.load(open(self.data_flds[tv],'rb'))
        
        for v in M.cat_vars: 
            self.data_flds[v] = data.Field(sequential=False, use_vocab=False)
        
        for v in M.contin_vars+M.range_vars+[M.dep]:
            self.data_flds[v] = data.Field(sequential=False, tensor_type=torch.FloatTensor, use_vocab=False)
        flds = [fld[0] for fld in self.data_flds.items()]
        for exs in run_distributed(get_examples, [[[f], flds] for f in self.preprocessed], M.cores):
            examples += exs
        return examples
    
    def process(self, tol=0.0):
        show(f'{self.split} iter: {self.iter}\n')
        self.preprocess()                                            # 45% time (40% in brand fix)

        if self.split=='train' and self.iter==1:
            for tv in M.text_vars: self.build_vocab(tv, M.combine[tv])   # 5% 
        self.numericalize_text()                                     # 40%
        self.postprocess_df()                                        # 10%
        missing = len(self.missing_idx)/len(self.idx)
        if missing<=tol or self.iter>2:
            if missing>tol and self.iter>2: 
                show(f'''{self.iter} iteration incomplete: {round(missing*100)}%!!
                                 Problem with the file system!''')
            return
        else:
            self.iter += 1
            return self.process(tol)
        #train.save(f'{SCRATCH}/train.obj')        
    
    def get_df(self, idxs=[]):
        files = self.preprocessed if len(idxs)==0 else [self.preprocessed[i] for i in idxs]
        df = pd.concat([read_df(f, processed=True) for f in files])    
        return df
    
    def save(self, file):
        pickle.dump(self, open(file,'wb'))

# Deinfitions for LM and Final Model

In [6]:
def to_gpu(x, *args, **kwargs):
    if USE_GPU:
        # is cuda available check kiils time/mem 
        return x.cuda(*args, **kwargs) if torch.cuda.is_available() else x
    else:
        return x

def T(a):
    if torch.is_tensor(a): res = a
    else:
        a = np.array(np.ascontiguousarray(a))
        if a.dtype in (np.int8, np.int16, np.int32, np.int64):
            res = torch.LongTensor(a.astype(np.int64))
        elif a.dtype in (np.float32, np.float64):
            return torch.FloatTensor(a.astype(np.float32))
        else: raise NotImplementedError
    return to_gpu(res, async=True)

# Making a Variable in torch invokes GPU kills time
def create_variable(x, volatile, requires_grad=False):
    if not isinstance(x, Variable):
        x = Variable(T(x), volatile=volatile, requires_grad=requires_grad)
    return x

def VV1_(x): return create_variable(x, True)     

def V1_(x, requires_grad=False):
    return create_variable(x, False, requires_grad=requires_grad)
    
def V1(x, requires_grad=False):
    return [V1(o, requires_grad) for o in x] if isinstance(x,list) else V1_(x, requires_grad)

def VV1(x):  return [VV1(o) for o in x] if isinstance(x,list) else VV1_(x)

def set_lrs(opt, lrs):
    if not isinstance(lrs, Iterable): lrs=[lrs]
    if len(lrs)==1: 
        lrs=lrs*len(opt.param_groups)
    elif len(lrs) != len(opt.param_groups):
        raise "Exception"
    for pg,lr in zip(opt.param_groups,lrs): pg['lr'] = lr
        

class StepperDist():
    def __init__(self, m, opt, crit, clip=0, reg_fn=None):
        self.m,self.opt,self.crit,self.clip,self.reg_fn = m,opt,crit,clip,reg_fn
        self.reset(True)

    def reset(self, train=True):
        if train: apply_leaf(self.m, set_train_mode)
        else: self.m.eval()
        if hasattr(self.m, 'reset'): self.m.reset()

    def step(self, xs, y):
        xtra = []
        output = self.m(*xs)
        if isinstance(output,(tuple,list)): output,*xtra = output
        self.opt.zero_grad()
        loss = raw_loss = self.crit(output, y)
        if self.reg_fn: loss = self.reg_fn(output, xtra, raw_loss)
        loss.backward()
        if is_dist(): apply_leaf(self.m, average_gradients)
        if self.clip:   # Gradient clipping
            nn.utils.clip_grad_norm(trainable_params_(self.m), self.clip)
        self.opt.step()
        return raw_loss.data[0]

    def evaluate(self, xs, y):
        preds = self.m(*xs)
        if isinstance(preds,(tuple,list)): preds=preds[0]
        return preds, self.crit(preds,y)


class DistLearner(Learner):
    def fit_gen(self, model, data, layer_opt, n_cycle, cycle_len=None, cycle_mult=1, cycle_save_name=None,
                metrics=None, callbacks=None, use_wd_sched=False, norm_wds=False, wds_sched_mult=None, **kwargs):
        if callbacks is None: callbacks=[]
        if metrics is None: metrics=self.metrics

        if use_wd_sched:
            # This needs to come before CosAnneal() because we need to read the initial learning rate from
            # layer_opt.lrs - but CosAnneal() alters the layer_opt.lrs value initially (divides by 100)
            if np.sum(layer_opt.wds) == 0:
                print('fit() warning: use_wd_sched is set to True, but weight decay(s) passed are 0. Use wds to '
                      'pass weight decay values.')
            batch_per_epoch = len(data.trn_dl)
            cl = cycle_len if cycle_len else 1
            self.wd_sched = WeightDecaySchedule(layer_opt, batch_per_epoch, cl, cycle_mult, n_cycle,
                                                norm_wds, wds_sched_mult)
            callbacks += [self.wd_sched]

        if cycle_len:
            cycle_end = self.get_cycle_end(cycle_save_name)
            cycle_batches = len(data.trn_dl)*cycle_len
            self.sched = CosAnneal(layer_opt, cycle_batches, on_cycle_end=cycle_end, cycle_mult=cycle_mult)
        elif not self.sched: self.sched=LossRecorder(layer_opt)
        callbacks+=[self.sched]
        for cb in callbacks: cb.on_train_begin()
        n_epoch = sum_geom(cycle_len if cycle_len else 1, cycle_mult, n_cycle)
        fit_dist(model, data, n_epoch, layer_opt.opt, self.crit,
            metrics=metrics, callbacks=callbacks, reg_fn=self.reg_fn, clip=self.clip, **kwargs)    
        
def fit_dist(model, data, epochs, opt, crit, metrics=None, callbacks=None, **kwargs):
    """ Fits a model

    Arguments:
       model (model): any pytorch module
           net = to_gpu(net)
       data (ModelData): see ModelData class and subclasses
       opt: optimizer. Example: opt=optim.Adam(net.parameters())
       epochs(int): number of epochs
       crit: loss function to optimize. Example: F.cross_entropy
    """
    stepper = StepperDist(model, opt, crit, **kwargs)
    metrics = metrics or []
    callbacks = callbacks or []
    avg_mom=0.98
    batch_num,avg_loss=0,0.

    for epoch in tnrange(epochs, desc='Epoch'):
        stepper.reset(True)
        t = tqdm(iter(data.trn_dl), leave=False, total=len(data.trn_dl))
        for (*x,y) in t:
            batch_num += 1
            loss = stepper.step(V1(x),V1(y))
            avg_loss = avg_loss * avg_mom + loss * (1-avg_mom)
            debias_loss = avg_loss / (1 - avg_mom**batch_num)
            t.set_postfix(loss=debias_loss)
            stop=False
            for cb in callbacks: stop = stop or cb.on_batch_end(debias_loss)
            if stop: return 

        vals = validate(stepper, data.val_dl, metrics)
        print(np.round([epoch, debias_loss] + vals, 6))
        stop=False
        for cb in callbacks: stop = stop or cb.on_epoch_end(vals)
        if stop: break

def validate(stepper, dl, metrics):
    loss,res = [],[]
    stepper.reset(False)
    for (*x,y) in iter(dl):
        preds,l = stepper.evaluate(VV1(x), VV1(y))
        loss.append(to_np(l))
        idx, x_txt, x_cat, x_cont = x
        preds = fix_range(to_np(preds).ravel(), x_cont[2], x_cont[3])
        res.append([f(preds,to_np(y)) for f in metrics])
    return [np.mean(loss)] + list(np.mean(np.stack(res),0))

def average_gradients(model):
    """ Gradient averaging. """
    size = float(dist.get_world_size())
    #show(f'In avg gradient : {get_rank()}')
    for param in model.parameters():
        if hasattr(param, 'grad') and hasattr(param.grad, 'data'):
            dist.all_reduce(param.grad.data, op=dist.reduce_op.SUM, group=0)
            param.grad.data /= size

def is_dist(): return dist._initialized

def get_rank(): return dist.get_rank() if dist._initialized else 0

def init_processes(rank, size, fn, backend='gloo'):
    """ Initialize the distributed environment. """
    os.environ['MASTER_ADDR'] = '127.0.0.1'
    os.environ['MASTER_PORT'] = '29500'
    dist.init_process_group(backend, rank=rank, world_size=size)
    fn(rank, size)
    
def torch_distributed(fn, size=2):
    processes = []
    for rank in range(size):
        p = Process(target=init_processes, args=(rank, size, fn))
        p.start()
        processes.append(p)

    for p in processes:
        p.join()
        
#mod = sys.modules['fastai.core']
#mod.V, mod.V_, mod.VV, mod.VV_, mod.T = V, V_, VV, VV_, T
#mod.create_variable, mod.to_gpu = create_variable, to_gpu
#sys.modules['fastai.core'] = mod
#from fastai.core import *

In [7]:
class MixedTextDataset(torchtext.data.Dataset):
    @staticmethod
    def sort_key(ex):   return len(ex.item_description)
    
# copy latest class definitions
class EmbeddingDropout(nn.Module):

    def __init__(self, embed):
        super().__init__()
        self.embed = embed

    def forward(self, words, dropout=0.1, scale=None):

        if dropout:
            size = (self.embed.weight.size(0),1)
            mask = Variable(dropout_mask(self.embed.weight.data, size, dropout))
            masked_embed_weight = mask * self.embed.weight
        else:
            masked_embed_weight = self.embed.weight

        if scale:
            masked_embed_weight = scale * masked_embed_weight

        padding_idx = self.embed.padding_idx

        if padding_idx is None:
            padding_idx = -1

        X = self.embed._backend.Embedding.apply(words,
                 masked_embed_weight, padding_idx, self.embed.max_norm,
                     self.embed.norm_type, self.embed.scale_grad_by_freq, self.embed.sparse)

        return X
    
class RNN_Encoder(nn.Module):

    initrange=0.1

    def __init__(self, bs, ntoken, emb_sz, nhid, nlayers, pad_token, bidir=False,
                 dropouth=0.3, dropouti=0.65, dropoute=0.1, wdrop=0.5):

        super().__init__()
        self.ndir = 2 if bidir else 1
        self.encoder = nn.Embedding(ntoken, emb_sz, padding_idx=pad_token)
        self.encoder_with_dropout = EmbeddingDropout(self.encoder)
        self.rnns = [nn.LSTM(emb_sz if l == 0 else nhid, (nhid if l != nlayers - 1 else emb_sz)//self.ndir,
             1, bidirectional=bidir, dropout=dropouth) for l in range(nlayers)]
        if wdrop: self.rnns = [WeightDrop(rnn, wdrop) for rnn in self.rnns]
        self.rnns = torch.nn.ModuleList(self.rnns)
        self.encoder.weight.data.uniform_(-self.initrange, self.initrange)

        self.bs,self.emb_sz,self.nhid,self.nlayers,self.dropoute = bs,emb_sz,nhid,nlayers,dropoute
        self.dropouti = LockedDropout(dropouti)
        self.dropouths = nn.ModuleList([LockedDropout(dropouth) for l in range(nlayers)])

    def forward(self, input):
        sl,bs = input.size()
        if bs!=self.bs:
            self.bs=bs
            self.reset()

        emb = self.encoder_with_dropout(input, dropout=self.dropoute if self.training else 0)
        emb = self.dropouti(emb)

        raw_output = emb
        new_hidden,raw_outputs,outputs = [],[],[]
        for l, (rnn,drop) in enumerate(zip(self.rnns, self.dropouths)):
            current_input = raw_output
            with warnings.catch_warnings():
                warnings.simplefilter("ignore")
                raw_output, new_h = rnn(raw_output, self.hidden[l])
            new_hidden.append(new_h)
            raw_outputs.append(raw_output)
            if l != self.nlayers - 1: raw_output = drop(raw_output)
            outputs.append(raw_output)

        self.hidden = repackage_var(new_hidden)
        return raw_outputs, outputs

    def one_hidden(self, l):
        nh = (self.nhid if l != self.nlayers - 1 else self.emb_sz)//self.ndir
        return Variable(self.weights.new(self.ndir, self.bs, nh).zero_(), volatile=not self.training)

    def reset(self):
        self.weights = next(self.parameters()).data
        self.hidden = [(self.one_hidden(l), self.one_hidden(l)) for l in range(self.nlayers)]


class MultiBatchRNN(RNN_Encoder):
    '''Either Takes first or last'''
    def __init__(self, max_sl, bptt, last=False, train_text=False, *args, **kwargs):
        self.max_sl,self.bptt, self.last, self.train_text = max_sl,bptt, last, train_text
        super().__init__(*args, **kwargs)

    def concat(self, arrs):
        return [torch.cat([l[si] for l in arrs]) for si in range(len(arrs[0]))]

    def forward(self, input):
        sl,bs = input.size()
        for l in self.hidden:
            for h in l: h.data.zero_()
        raw_outputs, outputs = [],[]
        max_sl = min(self.max_sl,sl)
        for i in range(0, max_sl, self.bptt):
            input_seq = input[i : min(i+self.bptt, max_sl)]
            r, o = super().forward(input_seq)
            #if (i<self.bptt*8) or i>(max_sl-self.bptt*8):
            #if i>(max_sl-self.bptt*16):
            if True:
                raw_outputs.append(r)
                outputs.append(o)
            if self.last==False: break

        return self.concat(raw_outputs), self.concat(outputs)

class LinearRNNOutput(nn.Module):
    initrange=0.1
    def __init__(self, n_out, nhid, dropout):
        super().__init__()
        self.decoder = nn.Linear(nhid, n_out)
        self.decoder.bias.data.fill_(0)
        self.decoder.weight.data.uniform_(-self.initrange, self.initrange)
        self.dropout = LockedDropout(dropout)

    def forward(self, input):
        raw_outputs, outputs = input
        output = self.dropout(outputs[-1])
        return output, raw_outputs, outputs

class SequentialRNNText(SequentialRNN):
    def forward(self, input):
        output, raw_outputs, outputs = super().forward(input)
        return (input, output), raw_outputs, outputs
    

class LinearDecoder(LinearRNNOutput):
    """ A custom Linear layer that reads the signals from the output of the RNN_Encoder layer,
    and decodes to a output of size n_tokens.
    """
    def __init__(self, n_out, nhid, dropout, tie_encoder=None):
        super().__init__(n_out, nhid, dropout)
        if tie_encoder: self.decoder.weight = tie_encoder.weight
       
    def forward(self, input):
        output, raw_outputs, outputs = super().forward(input)
        decoded = self.decoder(output.view(output.size(0)*output.size(1), output.size(2)))
        result = decoded.view(-1, decoded.size(1))
        return result, raw_outputs, outputs

    
def chunk_seq(x, max_seq, bptt, last=False, dim=1):
    x = x[:max_seq]
    x = x[-bptt:] if last else x[:bptt]
    return x.view(x.size(0)*x.size(1)) if dim==1 else x

class CrossEntropyPad(nn.CrossEntropyLoss):
    def __init__(self, bptt, max_seq, last=False, size_average=True):
        self.bptt, self.max_seq, self.last = bptt, max_seq, last
        super().__init__()
        
    def forward(self, input, target):
        _assert_no_grad(target)
        input_seq, result = input
        #chunk2d = partial(chunk_seq, max_seq=self.max_seq, bptt=self.bptt, last=self.last, dim=2)
        #print("INPUT_SEQ=", chunk2d(input_seq), "\nTARGET=", chunk2d(target))
        target = chunk_seq(target, self.max_seq, self.bptt, self.last)
        ret = F.cross_entropy(result, target, size_average=self.size_average)
        return ret

class RNNLearnerEmbedded(DistLearner):    
    def __init__(self, data, models, **kwargs):
        super().__init__(data, models, **kwargs)
        self.crit = F.cross_entropy

    def save_encoder(self, name): save_model(self.model[0], self.get_model_path(name))
    def load_encoder(self, name): load_model(self.model[0], self.get_model_path(name))
    

class MixedTextInputModel(nn.Module):
    def __init__(self, emb_szs, n_cont, emb_drop, out_sz, szs, drops,
                 y_range=None, use_bn=False, text_emb_sz=[], rnn_encoders=[]):
        super().__init__()
        
        self.embs = nn.ModuleList([nn.Embedding(c, s) for c,s in emb_szs])
        for emb in self.embs: emb_init(emb)
        n_emb = sum(e.embedding_dim for e in self.embs)
        self.n_emb, self.n_cont=n_emb, n_cont
        
        if not isinstance(rnn_encoders,list): 
            rnn_encoders = [rnn_encoders]
            text_emb_sz = [text_emb_sz]
        
        self.nrnn = 2
        self.rnn_bns = nn.ModuleList([nn.BatchNorm1d(self.nrnn*sz) for sz in text_emb_sz])
        
        self.text_emb_sz, self.rnn_encoders = text_emb_sz, nn.ModuleList(rnn_encoders)
        
        szs = [n_emb+n_cont+self.nrnn*sum(text_emb_sz)] + szs
        self.lins = nn.ModuleList([
            nn.Linear(szs[i], szs[i+1]) for i in range(len(szs)-1)])
        self.bns = nn.ModuleList([
            nn.BatchNorm1d(sz) for sz in szs[1:]])
        for o in self.lins: kaiming_normal(o.weight.data)
        self.outp = nn.Linear(szs[-1], out_sz)
        kaiming_normal(self.outp.weight.data)

        self.emb_drop = nn.Dropout(emb_drop)
        self.drops = nn.ModuleList([nn.Dropout(drop) for drop in drops])
        self.bn = nn.BatchNorm1d(n_cont)
        self.use_bn,self.y_range = use_bn,y_range

    def pool(self, x, bs, is_max):
        f = F.adaptive_max_pool1d if is_max else F.adaptive_avg_pool1d
        return f(x.permute(1,2,0), (1,)).view(bs,-1)
        
    def forward(self, idx, x_text, x_cat, x_cont):
        if self.n_emb != 0:
            x = [e(x_cat[i]) for i,e in enumerate(self.embs)]
            x = torch.cat(x, 1)
            x = self.emb_drop(x)

        y_min, y_max = x_cont[-2], x_cont[-1]
        y_min, y_max = y_min.view(y_min.size(0), 1), y_min.view(y_min.size(0), 1)
        if self.n_cont != 0:
            x_cont = torch.cat([xc.view(xc.size(0), 1) for xc in x_cont[:-2]], 1)
            x2 = self.bn(x_cont)
            x = torch.cat([x, x2], 1) if self.n_emb != 0 else x2
        
        for i,rnn_enc in enumerate(self.rnn_encoders):
            enc_raw_outputs, enc_outputs = rnn_enc(x_text[i])
            enc_last_layer = enc_outputs[-1]
            sl,bs,_ = enc_last_layer.size()
            #avgpool = self.pool(enc_last_layer, bs, False)
            mxpool = self.pool(enc_last_layer, bs, True)
            x3 = torch.cat([enc_last_layer[-1], mxpool], 1)
            #x3 = enc_last_layer[-1]
            x3 = self.rnn_bns[i](x3)
            x = torch.cat([x, x3], 1)
            
        for l,d,b in zip(self.lins, self.drops, self.bns):
            x = F.relu(l(x))
            if self.use_bn: x = b(x)
            x = d(x)
        x = self.outp(x)
        #if self.y_range: and bs will always come from rnn_encoders
        #x = F.sigmoid(x)
        #x = x*(y_max-y_min) 
        #x = x + y_min
        #self.pred = x
        # encoder outputs are needed by the regularization function
        return x, enc_raw_outputs, enc_outputs

    def reset(self, module=None):
        if not module: module = self
        for c in module.children():
            if hasattr(c, 'children'): MixedTextInputModel.reset(c) 
            if hasattr(c, 'reset'): c.reset()


class MixedTextLearner(DistLearner):
    def __init__(self, data, models, **kwargs):
        super().__init__(data, models, **kwargs)
        self.crit = RMSLELoss()
        # self.crit = F.cross_entropy

    def save_encoder(self, name, idx): save_model(self.model.rnn_encoders[idx], self.get_model_path(name))
    def load_encoder(self, name, idx): load_model(self.model.rnn_encoders[idx], self.get_model_path(name))
        
def _assert_no_grad(variable):
    assert not variable.requires_grad, \
        "nn criterions don't compute the gradient w.r.t. targets - please " \
        "mark these variables as volatile or not requiring gradients"
        
class RMSLELoss(nn.Module):
    def __init__(self, size_average=True):
        super().__init__()
        self.size_average = size_average
        
    def forward(self, input, target):
        _assert_no_grad(target)
        ret = F.mse_loss(input, target, size_average=self.size_average)**0.5
        return ret

class MixedTextDataLoader():
    def __init__(self, src, index, txt_flds, cat_flds, contin_flds, y_fld):
        self.src,self.index,self.txt_flds,self.cat_flds, self.contin_flds, self.y_fld = \
             src,index,txt_flds,cat_flds,contin_flds, y_fld
        self.train_txt = None
        self.bptt = None

    def __len__(self): return len(self.src)-1

    def __iter__(self):
        it = iter(self.src)
        for i in range(len(self)):
            b = next(it)
            if self.train_txt == None: 
                yield getattr(b, self.index), [getattr(b, x) for x in self.txt_flds], \
                    [getattr(b, x) for x in self.cat_flds], [getattr(b, x) for x in self.contin_flds], \
                    getattr(b, self.y_fld)
            else:
                x_txt = getattr(b, self.txt_flds[self.train_txt])
                yield x_txt[:-1], x_txt[1:]
            

class MixedTextModel(BasicModel):
    def get_layer_groups(self):
        # defines how the differential learning rates would be applied to model groups
        m=self.model
        mecs = m.rnn_encoders
        enc_embed = [e.encoder for e in mecs] +[e.encoder_with_dropout for e in mecs]
        enc_midl_layers = [e.rnns for e in mecs]+[e.dropouths for e in mecs]
        ip_layers = enc_midl_layers+[e for e in m.embs]+[m.bn]
        enc_op_layers = [e.dropouti for e in m.rnn_encoders]
        midl_layers = enc_op_layers + [m.rnn_bns] + children(m.lins)+children(m.bns)
        op_layer = m.outp
        return [enc_embed, ip_layers, midl_layers, op_layer]

class MixedTextData(ModelData):
    def __init__(self, path, trn_dl, val_dl, test_dl):
        self.path = path
        self.nts, self.pad_idxs, self.encoders = [], [], []
        super().__init__(path, trn_dl, val_dl, test_dl)
    
    def create_td(self, it): return TextDataLoader(it, self.text_fld, self.label_fld)
    
    def get_text_fld(self, tv):
        return pickle.load(open(self.text_fld_files[tv],'rb'))

    @classmethod
    def from_splits(cls, path, splits, data_flds, bs):
        '''cteates one split at a time only when needed
        files: need to be full set of files. Creates splits 
        tuple of size for distributed processing. size=1 for GPU.'''
            
        device = -1 if M.no_cuda else None
        iters = torchtext.data.BucketIterator.splits(splits, batch_size=bs, device=device)
        dls = [MixedTextDataLoader(it, M.index, M.text_vars, M.cat_vars, 
                                   M.contin_vars+M.range_vars, M.target) for it in iters]
        # can't have multiple dls per modeldata in fastai but can have multiple mds
        trn_dl, val_dl = dls[:2]
        test_dl = dls[2] if len(iters)==3 else None
        obj = cls.from_dls(path, trn_dl, val_dl, test_dl)
        
        for tv in M.text_vars:
            tf = data_flds[tv] 
            obj.nts.append(len(tf.vocab))
            obj.pad_idxs.append(tf.pad_token)
        return obj

    def text_learner(self, i, callbacks=[], get_learner=False):
        '''i is text_var index'''
        tie_weights=True
        text_var = M.text_vars[i]
        # val dl size is 1 always
        # trn_dl and val_dls are referenced by fit()
        #self.trn_dl, self.val_dl = self.trn_dls[rank], self.val_dls[0]
        #M.nbatch = int(0.2*min([len(md.trn_dl) for md in mds]))
        M.nbatch=1000*M.sample
        M.iter = 0 
        
        self.val_dl.train_txt = self.trn_dl.train_txt = i 
        self.val_dl.bptt = self.trn_dl.bptt = M.enc_bptt[i] 
        # ******WARNING******* bs argument is DIFFERENT is versions for RNN_Encoder bidir=False, after pad_ix
        train_text=True
        rnn_enc = MultiBatchRNN(M.enc_max_seq[i], M.enc_bptt[i], M.text_seq_last, train_text, M.bs, 
                                self.nts[i], M.text_emb_sz[i], M.enc_n_hid[i], M.enc_n_layers[i], 
                                pad_token=self.pad_idxs[i], bidir=False, dropouth=M.lm_dropouth, 
                                dropouti=M.lm_dropouti, dropoute=M.lm_dropoute, wdrop=M.lm_wdrop)
        enc = rnn_enc.encoder if tie_weights else None
        text_model = SequentialRNNText(rnn_enc, LinearDecoder(self.nts[i], M.text_emb_sz[i], 
                                                                    M.lm_dropout, tie_encoder=enc))
        if M.no_cuda == False: text_model = to_gpu(text_model)
        model = SingleModel(text_model)
        learner = RNNLearnerEmbedded(self, model, opt_fn=M.lm_opt_fn)
        learner.crit = CrossEntropyPad(bptt=M.enc_bptt[i], max_seq=M.enc_max_seq[i], last=M.text_seq_last)
        if M.ignore_pad: learner.crit.ignore_index = self.pad_idxs[i]
        learner.reg_fn = M.lm_reg_fn
        learner.clip=0.3
        
        if get_learner: return learner
        learner.fit(2.0e-2, 1, wds=1e-6, cycle_len=1, cycle_mult=2, callbacks=callbacks)
        # save learner in rank=0 process
        #if rank==0:
            #show(f'Process {rank} is saving encoder for {text_var}\n')
            #learner.save_encoder(f'{text_var}')
        #if get_learner: return learner
        return

    def model_mixed(self, cat_emb_szs, y_range_file, use_bn=False, callbacks=[], 
                    get_learner=False, model_text=False, **kwargs):
        
        M.nbatch = 5000*M.bs/128
        M.iter = 0 
        
        self.val_dl.train_txt = self.trn_dl.train_txt = None
        train_text = False
        if len(M.text_vars)!=0:
            # bidir=False, 
            rnn_encs = [MultiBatchRNN(M.enc_max_seq[i], M.enc_bptt[i], M.text_seq_last, 
                                  train_text, M.bs, self.nts[i], M.text_emb_sz[i], M.enc_n_hid[i], 
                                  M.enc_n_layers[i], pad_token=self.pad_idxs[i], bidir=False, dropouth=M.dropouth, 
                                  dropouti=M.dropouti, dropoute=M.dropoute, wdrop=M.wdrop) for i in range(len(M.text_vars))]
        
        # reconstruct y_rtange
        y_range = read_df(y_range_file)
        # recreate groupings
        y_group = y_range.groupby(M.cat_vars)
        y_range = pd.DataFrame({'min': y_group['min'].agg('min'), 'max': y_group['max'].agg('max')})
        
        out_sz = 1
        m = MixedTextInputModel(cat_emb_szs, len(M.contin_vars), M.cat_emb_drop, out_sz, M.fc_layers, 
                                M.fc_drops, y_range, use_bn, M.text_emb_sz, rnn_encs)
        
        if M.no_cuda == False: m = to_gpu(m)
        #model = BasicModel(m)
        model = MixedTextModel(m)
        learner = MixedTextLearner(self, model, opt_fn=M.opt_fn)
        learner.reg_fn = partial(seq2seq_reg, alpha=2, beta=1)
        learner.clip=0.3
        
        if len(M.text_vars)!= 0 and model_text==True:
            for i,tv in enumerate(M.text_vars): learner.load_encoder(f'{tv}', i)
        if get_learner: return learner
        learner.fit(M.lrs, 1, cycle_len=3, cycle_mult=1, cycle_save_name=f'mercari', callbacks=callbacks)
        if (rank==0): learner.save('mercari')
        return

In [8]:
def end_training(loss): 
    M.iter += 1
    if M.end_train: return True
    if M.iter%10==0: 
        t = (datetime.now()-M.start_time).seconds
        if t > 3300: M.end_train = True
    return False

def train_begin(): 
    M.end_train = False
    #M.start_time = datetime.now()
    M.iter = 0

cbs = Callback()
cbs.on_train_begin = train_begin
cbs.on_batch_end = end_training
cbs.on_epoch_end = end_training


def predict_structured(m, md, split, ref, index='id', recon=True):
    m.eval()
    if hasattr(m, 'reset'): m.reset()
    
    dl = getattr(md, f'{split}_dl')
    pred = 'pred_'+M.target
    cols = [[M.target, pred]+M.text_vars+M.cat_vars+M.contin_vars]
    df = pd.DataFrame(columns=cols)
    for *x,y in iter(dl):
        idx, xtxt, xcat, x_cont = x
        bdf = pd.DataFrame(columns=cols)
        for j,tv in enumerate(M.text_vars):
            xdf = pd.DataFrame(to_np(xtxt[j]).transpose())
            tf = ref.data_flds[tv]
            xdf = xdf.apply(lambda x: x.apply(lambda x1: tf.vocab.itos[x1]))
            bdf[tv] = xdf.apply(lambda x: '--'.join(x), axis=1)
        for j,cv in enumerate(M.cat_vars):
            cats = ['NaN']+list(ref.df_sample[cv].cat.categories)
            bdf[cv] = pd.Series(to_np(xcat[j])).apply(lambda x: cats[x])
        
        for i,cv in enumerate(M.contin_vars+M.range_vars):
            bdf[cv] = to_np(x_cont[i])
        bdf[index] = to_np(idx)
        bdf[M.dep] = to_np(y)
        y_pred = to_np(get_prediction(m(*VV1(x)))).ravel()
        bdf[pred] = fix_range(y_pred, x_cont[2], x_cont[3])
        df = df.append(bdf)
    df[index] = df[index].astype(int)
    df['diff'] = (df[M.target]-df[pred]).abs()
    #new_idx = split+'_'+M.index
    #df.rename(columns={M.index : new_idx}, inplace=True)
    df = df.sort_values('diff', ascending=False).set_index(M.index)
    return df[M.text_vars+M.cat_vars+M.contin_vars+M.range_vars+[M.target, 'pred_'+M.target, 'diff']]

def fix_range(y_pred, y_min, y_max):
    y_min = to_np(y_min)
    y_max = to_np(y_max)
    y_pred[y_pred<y_min] = y_min[y_pred<y_min]
    y_pred[y_pred>y_max] = y_max[y_pred>y_max]
    return y_pred
        
def create_submission(m, dl):
    index = 'test_id'
    cols = [index, M.target]
    df = pd.DataFrame(columns=cols)
    for *x,y in iter(dl):
        idx, x_txt, x_cat, x_cont = x
        bdf = pd.DataFrame(columns=cols)
        y_pred = to_np(get_prediction(m(*VV1(x)))).ravel()
        y_pred = fix_range(y_pred, x_cont[2], x_cont[3])
        bdf[M.target] = np.exp(y_pred)-1.0
        bdf[index] = to_np(idx).astype(int)
        df = df.append(bdf)
    df.sort_values(index, inplace=True)
    df.set_index(index, inplace=True)
    df.to_csv('sample_submission.csv')
    return 
    
def final_model(mds, cat_emb_szs, y_range_file, rank=0, size=1, model_text=False, 
                model_final=True, get_learner=False, use_bn=False): 
    
    mds[rank].create_dls()
    ret = []
    if model_text:
        for i, text_var in enumerate(M.text_vars):
            #print(f'Rank {get_rank()} : Modeling {text_var}...\n')
            ret.append(text_learner(mds, i, rank=rank, callbacks=[cbs], get_learner=get_learner))
    if model_final:
        ret.append(model_mixed(mds, cat_emb_szs, y_range_file, rank, callbacks=[cbs], use_bn=False, 
                               get_learner=get_learner, model_text=model_text))
    return ret     

In [9]:
PATH='data/'
IN = 'Input/'
SCRATCH = 'tmp/'
!mkdir -p {SCRATCH}
!ls {IN}

models		       train_nospcl_val20.csv
processed	       train_nospcl_val20_text20.csv
raw		       train.tsv
test_nospcl_val20.csv  val_nospcl_val20.csv
test.tsv	       val_nospcl_val20_text20.csv
tmp


## DataFrame Proc

In [10]:
%%time
train = ProcessDataFrame(f'{IN}/train.tsv','train', n=10)
train.process(tol=0.01)
gc.collect()
test = ProcessDataFrame(f'{IN}/test.tsv','test', ref=train, n=10)
test.process(tol=0.0)
gc.collect()

train iter: 1
Pre-processing train iter 1 (10)30% .30% .30% .30% .31% .30% .30% .31% .30% .31% . done
 done
 building vocab for item_description... .......... done
item_description tokens 6724
 building vocab for name... .......... done
name tokens 3134
Numericalizing ....................... done
Post processing  done
test iter: 1
Pre-processing test iter 1 (10)30% .31% .30% .30% .31% .30% .30% .31% .31% .30% . done
 done
Numericalizing ....................... done
Post processing  done
CPU times: user 2min 32s, sys: 4 s, total: 2min 36s
Wall time: 21min 8s


In [11]:
%%time
train_exs = train.get_examples()
s = int(len(train_exs)*0.9)
trn_examples, val_examples = train_exs[:s], train_exs[s:]
# data is already shuffled by sample operation
trn_ds = MixedTextDataset(trn_examples, train.data_flds)
val_ds = MixedTextDataset(val_examples, train.data_flds)
test_ds = MixedTextDataset(test.get_examples(), test.data_flds)
gc.collect()

tmp//train_13.csv        tmp//train_18.csv        tmp//train_12.csv        tmp//train_15.csv        tmp//train_14.csv        tmp//train_19.csv        tmp//train_16.csv        tmp//train_10.csv        tmp//train_11.csv        tmp//train_17.csv         done
tmp//test_14.csv        tmp//test_18.csv        tmp//test_16.csv        tmp//test_11.csv        tmp//test_13.csv        tmp//test_12.csv        tmp//test_17.csv        tmp//test_10.csv        tmp//test_15.csv        tmp//test_19.csv         done
CPU times: user 39.3 s, sys: 2.97 s, total: 42.3 s
Wall time: 3min 22s


In [19]:
M.bs, M.lrs= 256, 1e-2
M.enc_bptt=[40, 15]

md = MixedTextData.from_splits(SCRATCH, (trn_ds, val_ds, test_ds), train.data_flds, M.bs)
learner = md.model_mixed(train.cat_emb_szs, train.y_range, callbacks=[cbs], use_bn=False, 
                               get_learner=True, model_text=False)
learner.fit(1.0e-2, 1, wds=1e-6, cycle_len=2, cycle_mult=1, callbacks=[cbs])

A Jupyter Widget


  1%|          | 65/5208 [00:12<16:22,  5.24it/s, loss=1.75]


  2%|▏         | 130/5208 [00:23<15:22,  5.50it/s, loss=0.972]


  4%|▎         | 195/5208 [00:33<14:29,  5.77it/s, loss=0.768]


  5%|▍         | 257/5208 [00:44<14:23,  5.74it/s, loss=0.697]


...

 77%|███████▋  | 3996/5208 [14:16<04:19,  4.66it/s, loss=0.484]


 78%|███████▊  | 4057/5208 [14:30<04:06,  4.66it/s, loss=0.483]


 79%|███████▉  | 4118/5208 [14:43<03:53,  4.66it/s, loss=0.484]


 79%|███████▉  | 4134/5208 [14:47<03:50,  4.66it/s, loss=0.48]

KeyboardInterrupt: 

In [ ]:
learner.sched.plot_lr()

In [ ]:
create_submission(learner.model, md.test_dl)

In [ ]:
(datetime.now() - M.start_time).seconds

### Validate

In [ ]:
df_val = predict_structured(learner.model, md, 'val', train, index='id')

In [ ]:
pd.options.display.max_colwidth = 1000
df_val.head(5)